In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from sklearn.model_selection import train_test_split

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Libraries imported successfully!")


In [ ]:
# One-to-One RNN (Traditional Neural Network)
def create_one_to_one_model(input_dim, hidden_units=64, output_units=1):
    """
    Create a One-to-One model (essentially a feedforward network)
    """
    model = keras.Sequential([
        keras.layers.Dense(hidden_units, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(hidden_units//2, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(output_units, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create sample data for One-to-One classification
X_simple = np.random.randn(1000, 10)  # 1000 samples, 10 features
y_simple = (X_simple.sum(axis=1) > 0).astype(int)  # Binary classification

X_train_simple, X_test_simple, y_train_simple, y_test_simple = train_test_split(
    X_simple, y_simple, test_size=0.2, random_state=42
)

# Create and train One-to-One model
print("Creating One-to-One model...")
one_to_one_model = create_one_to_one_model(input_dim=10, hidden_units=32)

print("One-to-One Model architecture:")
one_to_one_model.summary()

history_simple = one_to_one_model.fit(
    X_train_simple, y_train_simple,
    batch_size=32,
    epochs=20,
    validation_data=(X_test_simple, y_test_simple),
    verbose=0
)

accuracy_simple = one_to_one_model.evaluate(X_test_simple, y_test_simple, verbose=0)[1]
print(f"One-to-One Model Accuracy: {accuracy_simple:.4f}")


In [ ]:
# One-to-Many RNN for Simple Text Generation
def create_one_to_many_model(vocab_size, embedding_dim=50, rnn_units=64, sequence_length=10):
    """
    Create a One-to-Many RNN for text generation
    """
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim),
        keras.layers.LSTM(rnn_units, return_sequences=True),
        keras.layers.LSTM(rnn_units),
        keras.layers.RepeatVector(sequence_length),
        keras.layers.LSTM(rnn_units, return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(vocab_size, activation='softmax'))
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create simple character-level data for text generation
def create_character_data():
    """Create simple character sequences for demonstration"""
    text = "hello world this is a simple text generation example"
    chars = sorted(list(set(text)))
    char_to_idx = {char: idx for idx, char in enumerate(chars)}
    idx_to_char = {idx: char for idx, char in enumerate(chars)}
    
    sequence_length = 5
    X = []
    y = []
    
    for i in range(len(text) - sequence_length):
        sequence = text[i:i+sequence_length]
        target = text[i+1:i+sequence_length+1]
        
        X.append([char_to_idx[char] for char in sequence])
        y.append([char_to_idx[char] for char in target])
    
    return np.array(X), np.array(y), char_to_idx, idx_to_char, chars

print("Creating character-level data...")
X_char, y_char, char_to_idx, idx_to_char, chars = create_character_data()

print(f"Character data shape: X={X_char.shape}, y={y_char.shape}")
print(f"Vocabulary size: {len(chars)}")
print(f"Characters: {chars}")

# Simple text generation function
def generate_text(model, start_string, char_to_idx, idx_to_char, num_generate=50):
    """Generate text using the trained model"""
    input_eval = [char_to_idx.get(s, 0) for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    # Low temperature for more predictable results
    temperature = 0.5
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # Use temperature to control randomness
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx_to_char.get(predicted_id, ''))
    
    return start_string + ''.join(text_generated)

# For demonstration, let's create a simpler sequence prediction model
def create_simple_sequence_model(vocab_size, sequence_length=5):
    """Create a simple sequence prediction model"""
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, 32, input_length=sequence_length),
        keras.layers.LSTM(64),
        keras.layers.Dense(vocab_size, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Prepare data for next character prediction
X_simple = X_char[:, :-1]  # All but last character
y_simple = X_char[:, -1]   # Last character

print(f"Simple prediction data: X={X_simple.shape}, y={y_simple.shape}")

# Create and train simple model
print("\nCreating simple character prediction model...")
simple_gen_model = create_simple_sequence_model(len(chars), sequence_length=4)

print("Simple generation model architecture:")
simple_gen_model.summary()

print("\nTraining simple generation model...")
gen_history = simple_gen_model.fit(
    X_simple, y_simple,
    batch_size=16,
    epochs=50,
    verbose=0
)

print(f"Final training accuracy: {gen_history.history['accuracy'][-1]:.4f}")

# Simple text generation
def simple_generate(model, seed, char_to_idx, idx_to_char, length=20):
    """Simple text generation"""
    result = seed
    current = seed
    
    for _ in range(length):
        # Prepare input
        if len(current) > 4:
            current = current[-4:]
        
        x_pred = np.array([[char_to_idx.get(c, 0) for c in current]])
        
        # Predict next character
        pred = model.predict(x_pred, verbose=0)
        next_idx = np.argmax(pred[0])
        next_char = idx_to_char.get(next_idx, '')
        
        result += next_char
        current += next_char
    
    return result

# Generate some text
seed = "hell"
generated = simple_generate(simple_gen_model, seed, char_to_idx, idx_to_char, length=30)
print(f"\nGenerated text from seed '{seed}': {generated}")

# Visualization
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(history_simple.history['accuracy'], label='Train')
plt.plot(history_simple.history['val_accuracy'], label='Validation')
plt.title('One-to-One Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.plot(gen_history.history['accuracy'])
plt.title('Text Generation Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.plot(gen_history.history['loss'])
plt.title('Text Generation Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

# Character frequency
plt.subplot(2, 2, 4)
char_counts = [len([c for c in generated if c == char]) for char in chars]
plt.bar(range(len(chars)), char_counts)
plt.title('Generated Character Frequency')
plt.xlabel('Character Index')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSummary of RNN Architectures:")
print("1. One-to-One: Traditional neural networks for fixed input-output")
print("2. One-to-Many: Text/sequence generation from fixed input")
print("3. Many-to-One: Sequence classification and sentiment analysis")
print("4. Many-to-Many: Sequence transformation and translation")
print("5. Each architecture serves specific use cases in sequence modeling")
